# Экспертные системы

В этом упражнении мы реализуем простейшую экспертную систему на основе продукционного представления знаний. В качестве примера будем создавать систему для определения животного на основе характерных признаков. Пример взят из книги **Экспертные системы. Принципы работы и примеры** / *ред. Р. Форсайт.* - М.: Радио и связь, 2009.

Знания в виде **дерева И-ИЛИ** представлены на рисунке:

![AND-OR Tree](images/and-or-tree.png)

## PyKnow для прямого вывода

Реализуем прямой вывод на базе библиотеки [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** - это библиотека для построения экспертных систем прямого вывода на Python, которая спроектирована так, чтобы быть похожей на классическую систему [CLIPS](http://www.clipsrules.net/index.html). 

Наивную систему прямого вывода тоже не составило бы труда реализовать самим, однако она была бы неэффективной. Для реализации эффективного матчинга правил и состояния рабочей памяти используется алгоритм *Rete*.

Для начала, установим PyKnow. Это можно сделать прямо из репозитория GitHub, чтобы получить самую последнюю версию:

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

In [1]:
from pyknow import *

Система в PyKnow определяется как класс, являющийся подклассом **KnowledgeEngine**. Каждое правило определяется отдельной функцией с аннотацией **@Rule**, которая указывает, когда должно срабатывать правило. Внутри правила мы можем добавить новые факты, используя функцию `declare`. Это приведет к тому, что механизм прямого вывода вызовет еще какие-то правила, пока прямой вывод не закончится.

In [3]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('острые зубы'),Fact('когти'),Fact('вперёд смотрящие глаза')),
           Fact('ест мясо')))
    def cornivor(self):
        self.declare(Fact('хищник'))
        
    @Rule(OR(Fact('волосы'),Fact('даёт молоко')))
    def mammal(self):
        self.declare(Fact('млекопитающее'))

    @Rule(Fact('млекопитающее'),
          OR(Fact('копыта'),Fact('жуёт жвачку')))
    def hooves(self):
        self.declare('копытное')
        
    @Rule(OR(Fact('оперение'),AND(Fact('летает'),Fact('откладывает яйца'))))
    def bird(self):
        self.declare('птица')
        
    @Rule(Fact('млекопитающее'),Fact('хищник'),
          Fact(color='рыжевато-коричневый'),
          Fact(pattern='тёмные пятна'))
    def monkey(self):
        self.declare(Fact(animal='обезьяна'))

    @Rule(Fact('млекопитающее'),Fact('хищник'),
          Fact(color='рыжевато-коричневый'),
          Fact(pattern='чёрные полосы'))
    def tiger(self):
        self.declare(Fact(animal='тигр'))

    @Rule(Fact('копытное'),
          Fact('длинная шея'),
          Fact('длинные ноги'),
          Fact(pattern='тёмные пятна'))
    def giraffe(self):
        self.declare(Fact(animal='жираф'))

    @Rule(Fact('копытное'),
          Fact(pattern='чёрные полосы'))
    def zebra(self):
        self.declare(Fact(animal='зебра'))

    @Rule(Fact('птица'),
          Fact('длинная шея'),
          Fact('не может летать'),
          Fact(color='чёрное с белым'))
    def straus(self):
        self.declare(Fact(animal='страус'))

    @Rule(Fact('птица'),
          Fact('плавает'),
          Fact('не может летать'),
          Fact(color='чёрное с белым'))
    def pinguin(self):
        self.declare(Fact(animal='пингвин'))

    @Rule(Fact('птица'),
          Fact('хорошо летает'))
    def albatros(self):
        self.declare(Fact(animal='альбатрос'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Животное - {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Как только мы определили базу знаний, мы заполняем нашу рабочую память некоторыми исходными данными. Для этого мы определили метод `factz`, который позволяет записать в рабочую память сразу несколько фактов.

Для запуска логического вывода мы вызываем метод run(). В результате в рабочую память добавляются новые предполагаемые факты, в том числе конечный факт, определяющий животное (если из исходных данных можно получить какой-то результат). В нашей системе также предусмотрено специальное правило с паттерн-матчингом - когда в рабочей памяти появится любое правило вида `animal=...`, сработает функция `print_result` и напечатает результат.

In [6]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='рыжевато-коричневый'),
    Fact(pattern='тёмные пятна'),
    Fact('острые зубы'),
    Fact('когти'),
    Fact('вперёд смотрящие глаза'),
    Fact('даёт молоко')])
ex1.run()

Животное - обезьяна


Можем также посмотреть на содержимое рабочей памяти:

In [5]:
ex1.facts

FactList([(0, InitialFact()),
          (1, Fact(color='рыжевато-коричневый')),
          (2, Fact(pattern='чёрные полосы')),
          (3, Fact('острые зубы')),
          (4, Fact('когти')),
          (5, Fact('вперёд смотрящие глаза')),
          (6, Fact('даёт молоко')),
          (7, Fact('млекопитающее')),
          (8, Fact('хищник')),
          (9, Fact(animal='тигр'))])

## Обратный вывод

Попробуем реализовать простую экспертную систему обратного вывода самостоятельно. Для представления правил и запросов определим несколько классов:
- **Ask** - вопрос, который необходимо задать пользователю. Также содержит набор возможных ответов
- **If** - правило или просто синтаксический сахар для хранения содержимого правила
- **AND/OR** - классаы для представления ветвей дерева с пометкой И/ИЛИ. Они хранят список аргументов. Для упрощения кода вся функциональность определена в родительском классе **Content**

In [7]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x))
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices))
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Правила базы знаний нашей системы можно записать как список фактов в виде пар атрибут-значение. Они представляют собой один большой словарь, который сопоставляет действия (новые факты, которые должны быть вставлены в рабочую память) с условиями, выраженными в виде выражений И-ИЛИ.

In [8]:
rules = {
    'default': Ask(['y','n']),
    'цвет' : Ask(['рыжевато-коричневый','чёрное с белым','другое']),
    'узор' : Ask(['чёрные полосы','тёмные пятна']),
    'млекопитающее': If(OR(['волосы','даёт молоко'])),
    'хищник': If(OR([AND(['острые зубы','когти','вперед смотрящие глаза']),'ест мясо'])),
    'копытное': If(['млекопитающее',OR(['копыта','жуёт жвачку'])]),
    'птица': If(OR(['оперение',AND(['летает','откладывает яйца'])])),
    'животное:обезьяна' : If(['млекопитающее','хищник','цвет:рыжевато-коричневый','узор:тёмные пятна']),
    'животное:тигр' : If(['млекопитающее','хищник','цвет:рыжевато-коричневый','узор:чёрные полосы']),
    'животное:жираф' : If(['копытное','длинная шея','длинные ноги','узор:тёмные пятна']),
    'животное:зебра' : If(['копытное','узор:чёрные полосы']),
    'животное:страус' : If(['птица','длинная шея','цвет:черное с белым','не может летать']),
    'животное:пингвин' : If(['птица','плавает','цвет:черное с белым','не может летать']),
    'животное:альбатрос' : If(['птица','хорошо летает'])
}

Конструкция в левой части (ключ словаря) - может быть либо именем атрибута (`хищник`, в этом случае подразумевается логическое значение да/нет), либо составной конструкцией вида `атрибут:значеие`, например `животное:пингвин`.

Сама по себе база знаний - это словарь, а конструкции `If`, `AND`, `Ask` и др. - это синтаксические конструкции, позволяющие определить язык представления знаний.

Чтобы выполнить обратный вывод, необходимо определить класс **Knowledgebase**, содержащий в себе всю логику логического вывода. Он также будет включать в себя:

- **Working memory** — словарь, сопоставляющий ключи со значениями
- **rules** - правила базы знаний в формате, определенном выше

Два основных метода:

**get**, чтобы получить значение ключа. Например, get('color') вернет значение цветового слота. Если мы запросим get('color:blue'), он запросит цвет, а затем вернет значение y/n (да/нет) в зависимости от цвета.
**eval** выполняет фактический вывод, т. е. проходит по дереву И/ИЛИ, оценивает подцели и т. д.

Наконец опишем рекурсивую машину обратного вывода:

In [9]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Теперь давайте проведем консультацию, определив нашу базу знаний о животных. Обратите внимание, что вам будут задавать вопросы. Вы можете ответить, набрав y/n для вопросов типа «да-нет» или указав номер (0..N) для вопросов с более длинными вариантами ответов.

In [11]:
kb = KnowledgeBase(rules)
kb.get('животное')

волосы
y/n
y
острые зубы
y/n
n
ест мясо
y/n
n
хищник
y/n
n
копыта
y/n
y
длинная шея
y/n
y
длинные ноги
y/n
y
узор
0. чёрные полосы
1. тёмные пятна
0


'зебра'

### Заключение

Мы привели простейщий механизм логического вывода. Вы можете попробовать усовершенствовать его, добавив вероятностные рассуждения, или реализовав смешанный логический вывод.

Как видите, сама по себе реализация процесса вывода не слишком сложна (хотя, если стремиться к эффективной реализации, например реализуюя алгорит **Retè** для прямого вывода, код может существенно усложниться). Основную сложность представляет создание достаточно большой, правдоподобной и непротиворечивой **базы знаний** - в рамках учебного примера мы не можем уделилить этому достаточно внимания.